### Testing walks-policy all vs mask

In [ ]:
# Running standard walks-policy mask (WW for complex walks)
!pairtools parse \
  /home/agalicina/tmp/pairtools_parse_update/data/arima3_link2019/C57_BMDM_HiC.lane1.mm10.sample1mln.bam \
  -c /home/agalicina/GENOMES/distiller_mm10_kit/mm10.reduced.chrom.sizes \
  -o ./arima_1mln_test_mask.pairsam \
  --assembly mm10 \
  --min-mapq 1 \
  --max-molecule-size 500 \
  --drop-seq --drop-sam \
  --add-columns pos3,pos5,read_len \
  --max-inter-align-gap 20 \
  --no-flip \
  --walks-policy mask

In [ ]:
# Running walks-policy all
!pairtools parse \
  /home/agalicina/tmp/pairtools_parse_update/data/arima3_link2019/C57_BMDM_HiC.lane1.mm10.sample1mln.bam \
  -c /home/agalicina/GENOMES/distiller_mm10_kit/mm10.reduced.chrom.sizes \
  -o ./arima_1mln_test_all.pairsam \
  --assembly mm10 \
  --min-mapq 1 \
  --max-molecule-size 500 \
  --drop-seq --drop-sam \
  --add-columns pos3,pos5,read_len \
  --max-inter-align-gap 20 \
  --no-flip \
  --walks-policy all

Reading and comparing the parsed pair-files:

In [10]:
import pandas as pd

In [12]:
# Reading the masked file
df_masked = pd.read_csv('./arima_1mln_test_mask.pairsam', sep='\t', comment='#', header=None)
df_masked.columns = 'readID chrom1 pos1 chrom2 pos2 strand1 strand2 pair_type pos51 pos52 pos31 pos32 read_len1 read_len2'.split()

In [13]:
# Reading the file with all rescued chimeras
df_all = pd.read_csv('./arima_1mln_test_all.pairsam', sep='\t', comment='#', header=None)
df_all.columns = 'readID chrom1 pos1 chrom2 pos2 strand1 strand2 pair_type pos51 pos52 pos31 pos32 read_len1 read_len2'.split()

In [24]:
ww_ids = df_masked.query('pair_type=="WW"')['readID'].values
df_rescued = df_all.set_index('readID').loc[ww_ids,:].reset_index()
df_rescued_valid = df_rescued.query('pair_type in ["PP", "JJ"]')

In [42]:
# Number of WW in the initial dataset, percentage out of initial pairsam file
len(ww_ids), 100*len(ww_ids)/len(df_masked)

(71543, 7.1539923783277315)

In [34]:
df_rescued[:5]

,readID,chrom1,pos1,chrom2,pos2,strand1,strand2,pair_type,pos51,pos52,pos31,pos32,read_len1,read_len2
0,SRR6660238.10,chr3,96261298,chr3,96251347,+,+,PP,96261342,96251447,96261298,96251347,100,101
1,SRR6660238.10,chr3,96586745,chr3,96261298,+,+,JJ,96586799,96261342,96586745,96261298,100,100
2,SRR6660238.23,chr9,71972700,chr10,91862052,+,-,PP,71972733,91861952,71972700,91862052,101,101
3,SRR6660238.23,chr10,74605170,chr9,71972700,-,+,JJ,74605104,71972733,74605170,71972700,101,101
4,SRR6660238.59,chr17,60390189,!,0,-,-,PM,60390089,0,60390189,0,101,101


In [43]:
# Number of pairs retrieved from all WW, percentage out of initial pairsam file
len(df_rescued_valid), 100*len(df_rescued_valid)/len(df_masked)

(46094, 4.6092018043224146)

In [44]:
# Number of reads rescued with rescue_complex_walk 
# (Note the difference from number of pairs)
# percentage out of all WW, percentage out of initial pairsam file

len(np.unique(df_rescued_valid.readID)), \
100*len(np.unique(df_rescued_valid.readID))/len(ww_ids), \
100*len(np.unique(df_rescued_valid.readID))/len(df_masked)

(28450, 39.766294396377006, 2.844877670260179)

### Dependence on MAPQ: setting MAPQ>=30 reduces the number of rescuable pairs

In [50]:
# Running standard walks-policy mask (WW for complex walks)
!pairtools parse \
  /home/agalicina/tmp/pairtools_parse_update/data/arima3_link2019/C57_BMDM_HiC.lane1.mm10.sample1mln.bam \
  -c /home/agalicina/GENOMES/distiller_mm10_kit/mm10.reduced.chrom.sizes \
  -o ./arima_1mln_test_mask_MAPQ30.pairsam \
  --assembly mm10 \
  --min-mapq 30 \
  --max-molecule-size 500 \
  --drop-seq --drop-sam \
  --add-columns pos3,pos5,read_len \
  --max-inter-align-gap 20 \
  --no-flip \
  --walks-policy mask

In [49]:
# Running walks-policy all
!pairtools parse \
  /home/agalicina/tmp/pairtools_parse_update/data/arima3_link2019/C57_BMDM_HiC.lane1.mm10.sample1mln.bam \
  -c /home/agalicina/GENOMES/distiller_mm10_kit/mm10.reduced.chrom.sizes \
  -o ./arima_1mln_test_all_MAPQ30.pairsam \
  --assembly mm10 \
  --min-mapq 30 \
  --max-molecule-size 500 \
  --drop-seq --drop-sam \
  --add-columns pos3,pos5,read_len \
  --max-inter-align-gap 20 \
  --no-flip \
  --walks-policy all

Reading and comparing the parsed pair-files:

In [51]:
# Reading the masked file
df_masked = pd.read_csv('./arima_1mln_test_mask_MAPQ30.pairsam', sep='\t', comment='#', header=None)
df_masked.columns = 'readID chrom1 pos1 chrom2 pos2 strand1 strand2 pair_type pos51 pos52 pos31 pos32 read_len1 read_len2'.split()

In [52]:
# Reading the file with all rescued chimeras
df_all = pd.read_csv('./arima_1mln_test_all_MAPQ30.pairsam', sep='\t', comment='#', header=None)
df_all.columns = 'readID chrom1 pos1 chrom2 pos2 strand1 strand2 pair_type pos51 pos52 pos31 pos32 read_len1 read_len2'.split()

In [53]:
ww_ids = df_masked.query('pair_type=="WW"')['readID'].values
df_rescued = df_all.set_index('readID').loc[ww_ids,:].reset_index()
df_rescued_valid = df_rescued.query('pair_type in ["PP", "JJ"]')

In [54]:
# Number of WW in the initial dataset, percentage out of initial pairsam file
len(ww_ids), 100*len(ww_ids)/len(df_masked)

(85074, 8.507034197529507)

In [56]:
# Number of pairs retrieved from all WW, percentage out of initial pairsam file
len(df_rescued_valid), 100*len(df_rescued_valid)/len(df_masked)

(38971, 3.896932431905428)

In [57]:
# Number of reads rescued with rescue_complex_walk 
# (Note the difference from number of pairs)
# percentage out of all WW, percentage out of initial pairsam file

len(np.unique(df_rescued_valid.readID)), \
100*len(np.unique(df_rescued_valid.readID))/len(ww_ids), \
100*len(np.unique(df_rescued_valid.readID))/len(df_masked)

(25033, 29.42497120154219, 2.5031923627284027)

### Dependence on MAPQ: setting MAPQ>=0 rescues nearly everything

In [58]:
# Running standard walks-policy mask (WW for complex walks)
!pairtools parse \
  /home/agalicina/tmp/pairtools_parse_update/data/arima3_link2019/C57_BMDM_HiC.lane1.mm10.sample1mln.bam \
  -c /home/agalicina/GENOMES/distiller_mm10_kit/mm10.reduced.chrom.sizes \
  -o ./arima_1mln_test_mask_MAPQ0.pairsam \
  --assembly mm10 \
  --min-mapq 0 \
  --max-molecule-size 500 \
  --drop-seq --drop-sam \
  --add-columns pos3,pos5,read_len \
  --max-inter-align-gap 20 \
  --no-flip \
  --walks-policy mask

In [66]:
# Running walks-policy all
!pairtools parse \
  /home/agalicina/tmp/pairtools_parse_update/data/arima3_link2019/C57_BMDM_HiC.lane1.mm10.sample1mln.bam \
  -c /home/agalicina/GENOMES/distiller_mm10_kit/mm10.reduced.chrom.sizes \
  -o ./arima_1mln_test_all_MAPQ0.pairsam \
  --assembly mm10 \
  --min-mapq 0 \
  --max-molecule-size 500 \
  --drop-seq --drop-sam \
  --add-columns pos3,pos5,read_len \
  --max-inter-align-gap 20 \
  --no-flip \
  --walks-policy all

Reading and comparing the parsed pair-files:

In [67]:
# Reading the masked file
df_masked = pd.read_csv('./arima_1mln_test_mask_MAPQ0.pairsam', sep='\t', comment='#', header=None)
df_masked.columns = 'readID chrom1 pos1 chrom2 pos2 strand1 strand2 pair_type pos51 pos52 pos31 pos32 read_len1 read_len2'.split()

In [68]:
# Reading the file with all rescued chimeras
df_all = pd.read_csv('./arima_1mln_test_all_MAPQ0.pairsam', sep='\t', comment='#', header=None)
df_all.columns = 'readID chrom1 pos1 chrom2 pos2 strand1 strand2 pair_type pos51 pos52 pos31 pos32 read_len1 read_len2'.split()

In [69]:
ww_ids = df_masked.query('pair_type=="WW"')['readID'].values
df_rescued = df_all.set_index('readID').loc[ww_ids,:].reset_index()
df_rescued_valid = df_rescued.query('pair_type in ["PP", "JJ"]')

In [70]:
# Number of WW in the initial dataset, percentage out of initial pairsam file
len(ww_ids), 100*len(ww_ids)/len(df_masked)

(73992, 7.398881848080532)

In [71]:
# Number of pairs retrieved from all WW, percentage out of initial pairsam file
len(df_rescued_valid), 100*len(df_rescued_valid)/len(df_masked)

(142032, 14.202589288660588)

In [72]:
# Number of reads rescued with rescue_complex_walk 
# (Note the difference from number of pairs)
# percentage out of all WW, percentage out of initial pairsam file

len(np.unique(df_rescued_valid.readID)), \
100*len(np.unique(df_rescued_valid.readID))/len(ww_ids), \
100*len(np.unique(df_rescued_valid.readID))/len(df_masked)

(73495, 99.32830576278516, 7.349183985088641)